## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-01-10-49-44 +0000,bbc,"Andrew should answer Epstein questions in US, ...",https://www.bbc.com/news/articles/c3dnnpvjkjvo...
1,2025-11-01-10-33-55 +0000,nyt,Future of Gerrymandering? Here’s How Weird Thi...,https://www.nytimes.com/2025/11/01/upshot/gerr...
2,2025-11-01-10-19-38 +0000,nyt,Quality Time,https://www.nytimes.com/2025/11/01/briefing/qu...
3,2025-11-01-10-07-47 +0000,bbc,What do you think about the cost of renting?,https://www.bbc.com/news/videos/cq500jyvwpyo?a...
4,2025-11-01-10-00-56 +0000,wapo,Trump’s China deal eases U.S. rare earths plig...,https://www.washingtonpost.com/business/2025/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,39
112,new,17
220,he,14
49,snap,13
34,shutdown,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
262,2025-10-31-14-56-14 +0000,nypost,Trump’s call to go ‘nuclear’ to end government...,https://nypost.com/2025/10/31/us-news/trumps-c...,90
119,2025-10-31-23-04-00 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...,90
233,2025-10-31-16-22-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/filibuster...,89
203,2025-10-31-18-14-31 +0000,nypost,Judges order Trump admin to use emergency fund...,https://nypost.com/2025/10/31/us-news/federal-...,79
167,2025-10-31-20-22-22 +0000,bbc,US judges say Trump administration must contin...,https://www.bbc.com/news/articles/cr433x9zqq4o...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
262,90,2025-10-31-14-56-14 +0000,nypost,Trump’s call to go ‘nuclear’ to end government...,https://nypost.com/2025/10/31/us-news/trumps-c...
109,60,2025-10-31-23-24-00 +0000,wsj,The White House is imposing new restrictions o...,https://www.wsj.com/politics/policy/white-hous...
118,47,2025-10-31-23-06-43 +0000,nypost,Dems cynically keep government shut down past ...,https://nypost.com/2025/10/31/us-news/dems-cyn...
297,46,2025-10-31-11-41-06 +0000,nypost,Prince Andrew thrown ‘under the bus’ by royal ...,https://nypost.com/2025/10/31/world-news/princ...
35,42,2025-11-01-09-00-32 +0000,nyt,He Stayed in Belarus for His Imprisoned Wife. ...,https://www.nytimes.com/2025/11/01/world/europ...
165,40,2025-10-31-20-26-00 +0000,wsj,The federal government pays out $100 billion a...,https://www.wsj.com/politics/policy/what-to-kn...
220,36,2025-10-31-17-13-09 +0000,bbc,"Canada's Carney to visit Xi in China, marking ...",https://www.bbc.com/news/articles/cx27pe8ekpdo...
47,34,2025-11-01-07-30-05 +0000,nypost,California dad stabbed on way to Halloween fes...,https://nypost.com/2025/11/01/us-news/californ...
245,33,2025-10-31-16-00-00 +0000,wsj,Hate Giant Airports? You Can Get Help Navigati...,https://www.wsj.com/lifestyle/travel/airport-v...
276,31,2025-10-31-13-47-58 +0000,nypost,Video shows deranged Zohran Mamdani passionate...,https://nypost.com/2025/10/31/us-news/mamdani-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
